<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Segmenting-and-Clustering-Neighborhoods-in-Toronto-part-1" data-toc-modified-id="Segmenting-and-Clustering-Neighborhoods-in-Toronto-part-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Segmenting and Clustering Neighborhoods in Toronto part 1</a></span><ul class="toc-item"><li><span><a href="#scrape-with-BeautifulSoup" data-toc-modified-id="scrape-with-BeautifulSoup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>scrape with BeautifulSoup</a></span></li><li><span><a href="#Data-preprocessing-and-cleaning" data-toc-modified-id="Data-preprocessing-and-cleaning-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data preprocessing and cleaning</a></span></li></ul></li><li><span><a href="#Segmenting-and-Clustering-Neighborhoods-in-Toronto-part-2" data-toc-modified-id="Segmenting-and-Clustering-Neighborhoods-in-Toronto-part-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Segmenting and Clustering Neighborhoods in Toronto part 2</a></span></li><li><span><a href="#Segmenting-and-Clustering-Neighborhoods-in-Toronto-part-3" data-toc-modified-id="Segmenting-and-Clustering-Neighborhoods-in-Toronto-part-3-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Segmenting and Clustering Neighborhoods in Toronto part 3</a></span><ul class="toc-item"><li><span><a href="#Cluster-Neighborhoods-with-KMeans" data-toc-modified-id="Cluster-Neighborhoods-with-KMeans-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Cluster Neighborhoods with KMeans</a></span></li></ul></li></ul></div>

# Segmenting and Clustering Neighborhoods in Toronto part 1

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

##  scrape with BeautifulSoup

In [3]:
Bsoup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'xml')

In [4]:
scrape=Bsoup.find('table')

In [5]:
# define the dataframe columns
column_names = ['Postalcode','Borough','Neighborhood']

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

In [6]:
# Then let's loop through the data and fill the dataframe one row at a time.
for tr in scrape.find_all('tr'):
    data=[]
    for td in tr.find_all('td'):
        data.append(td.text.strip())
    if len(data)==3:
        df.loc[len(df)] = data

In [7]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data preprocessing and cleaning

In [8]:
# Drop the rows 'Not assigned' in  Borough 
df = df[df.Borough != 'Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# gropuby the neighbourhoods with same Postalcode
df = df.groupby(['Postalcode','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
# Replacing neighbourhoods 'Not assigned' with names of Borough
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
# shape
df.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto part 2

In [12]:
# get the latitude and the longitude coordinates of each neighborhood. 

In [13]:
Geocoder = pd.read_csv('https://cocl.us/Geospatial_data')
Geocoder.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# merge geocoder dataframe with latitude and longitude with the df with Borough and Neighborhood
df = pd.merge(df,Geocoder,left_on='Postalcode', right_on='Postal Code')

In [15]:
df= df.drop('Postal Code', axis=1)

In [16]:
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Segmenting and Clustering Neighborhoods in Toronto part 3

In [17]:
# work with only boroughs that contain the word Toronto 
df_toronto = df[df['Borough'].str.contains('Toronto',regex=False)]
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [18]:
# Generate maps with Folium
import folium

In [19]:
Toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(Toronto)
Toronto

<img src="map.jpg">

##  Cluster Neighborhoods with KMeans

In [20]:
from sklearn.cluster import KMeans

In [21]:
# set number of clusters
kclusters = 5

cluster_toronto = df_toronto.drop(['Postalcode','Borough','Neighborhood'],1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(cluster_toronto)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 1, 0, 0, 3, 0, 2, 0, 3, 1, 0, 3, 1, 0, 1, 4, 4, 4, 4,
       2, 4, 3, 2, 4, 3, 2, 4, 3, 4, 0, 0, 0, 0, 0, 0, 1])

In [22]:
df_toronto['Cluster Labels'] = kmeans.labels_

C:\Users\rosario\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_toronto.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,2


In [24]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [25]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<img src="cluster.jpg">